In [30]:
from llama import Dialog, Llama
import os
import torch
from typing import List, Optional
import datetime

os.environ["HUGGINGFACE_CO_RESOLVE_ENDPOINT"] = "https://hf-mirror.com"
os.environ["HF_ENDPOINT "] = "https://hf-mirror.com"


print(torch.cuda.is_available())
print(torch.backends.cudnn.is_available())
print(torch.cuda_version)
print(torch.backends.cudnn.version())

os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12355'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"

True
True
12.1
8801


In [2]:
    check_point_directory = "Meta-Llama-3-8B-Instruct"
    tokenizer_path = "Meta-Llama-3-8B-Instruct/tokenizer.model"
    temperature = 0.6  # How much randomness is included in model output
    top_p = 0.9  # Top-p probability threshold for nucleus sampling
    max_sequence_length = 8192  # The llama 3 model model produces up to 8192 tokens so max sequence length needs to be <= 8192
    max_batch_size = 4
    model_parallel_size = 1

In [3]:
    if not torch.distributed.is_initialized():
        torch.distributed.init_process_group(backend='gloo')

In [4]:
    # Initialize the model
    generator = Llama.build(
        ckpt_dir=check_point_directory,
        tokenizer_path=tokenizer_path,
        max_seq_len=max_sequence_length,
        max_batch_size=max_batch_size,
        model_parallel_size=model_parallel_size
    )


> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


C:\anaconda3\envs\llama3-310\lib\site-packages\torch\__init__.py:747: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:433.)
  _C._set_default_tensor_type(t)


Loaded in 64.02 seconds


In [5]:
    dialogs: List[Dialog] = [
        [{"role": "user", "content": "what is the recipe of mayonnaise?"}],
        [
            {"role": "user", "content": "I am going to Paris, what should I see?"},
            {
                "role": "assistant",
                "content": """\
    Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

    1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
    2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
    3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

    These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.""",
            },
            {"role": "user", "content": "What is so great about #1?"},
        ],
        [
            {"role": "system", "content": "Always answer with Haiku"},
            {"role": "user", "content": "I am going to Paris, what should I see?"},
        ],
        [
            {
                "role": "system",
                "content": "Always answer with emojis",
            },
            {"role": "user", "content": "How to go from Beijing to NY?"},
        ],
    ]

In [6]:
    results = generator.chat_completion(
        dialogs,
        max_gen_len=max_sequence_length,
        temperature=temperature,
        top_p=top_p,
    )


In [7]:
    for dialog, result in zip(dialogs, results):
        for msg in dialog:
            print(f"{msg['role'].capitalize()}: {msg['content']}\n")
        print(
            f"> {result['generation']['role'].capitalize()}: {result['generation']['content']}"
        )
        print("\n==================================\n")

User: what is the recipe of mayonnaise?

> Assistant: The classic condiment! Mayonnaise is a thick, creamy sauce made from a combination of oil, egg yolks, acid (such as vinegar or lemon juice), and seasonings. Here's a simple recipe for homemade mayonnaise:

**Ingredients:**

* 2 large egg yolks
* 1 tablespoon lemon juice or white vinegar
* 1/2 teaspoon Dijon mustard (optional)
* 1/2 cup (120 ml) neutral-tasting oil, such as canola, grapeseed, or sunflower oil
* Salt, to taste

**Instructions:**

1. **In a medium-sized bowl**, whisk together the egg yolks, lemon juice or vinegar, and Dijon mustard (if using) until well combined.
2. **Slowly add the oil** while continuously whisking the mixture. Start with a very slow, thin stream and gradually increase the flow as the mixture thickens.
3. **Whisk constantly** to ensure the oil emulsifies with the egg yolks. You may need to stop and scrape down the sides of the bowl a few times to ensure everything gets fully incorporated.
4. **Continu

In [5]:
def dialog_chat(dialogs_temp:[Dialog]):
    results = generator.chat_completion(
        dialogs_temp,
        max_gen_len=max_sequence_length,
        temperature=temperature,
        top_p=top_p,
    )
    for dialog, result in zip(dialogs_temp, results):
        for msg in dialog:
            print(f"{msg['role'].capitalize()}: {msg['content']}\n")
        print(
            f"> {result['generation']['role'].capitalize()}: {result['generation']['content']}"
        )
        print("\n==================================\n")
    

In [6]:
def dialog_chat_user(content: str):
    dialogs_temp:[Dialog] = [ [{"role": "user", "content": content}]]
    dialog_chat(dialogs_temp)
    

In [7]:
dialog_chat_user("2+2=?")

User: 2+2=?

> Assistant: The answer is 4!




In [36]:
dialogs_temp: List[Dialog] = [
        [
            {"role": "cmdb", "content": "RTM_PRODUCT config"},
            {
                "role": "assistant",
                "content": """RTM_PRODUCT config:
                
  "key": "RTM_PRODUCT",
  "name": "RTM产品线",
  "description": "RTM_PRODUCT",
  "attrValid": null,
  "filter": {},
  "metaSchema": {
    "unique_key_rule": [
      "platform",
      "account",
      "resourceInstanceId"
    ],
    "icon": "type1",
    "autoKey": "id",
    "subscribeConfig": {
      "subscribeItems": [],
      "isPublish": false
    },
    "orgAuthorization": {
      "enable": false
    },
    "required": [
      "account",
      "companyId",
      "platform",
      "regionId",
      "resourceInstanceId",
      "name"
    ],
    "properties": {
      "resourceInstanceId": {
        "typeKey": "string",
        "is_relation": "N",
        "rangeLength": "",
        "name": "唯一实例ID",
        "pattern": "",
        "typeName": "字符型",
        "default_value": "",
        "id": "resourceInstanceId",
        "type": "string",
        "skw": false,
        "properties": {}
      },
      "instanceName": {
        "typeKey": "string",
        "is_relation": "N",
        "rangeLength": "",
        "name": "实例名称",
        "pattern": "",
        "typeName": "字符型",
        "default_value": "",
        "id": "instanceName",
        "type": "string",
        "skw": false,
        "properties": {}
      },
      "instanceType": {
        "typeKey": "string",
        "is_relation": "N",
        "rangeLength": "",
        "name": "主机类型",
        "pattern": "",
        "typeName": "字符型",
        "default_value": "",
        "id": "instanceType",
        "type": "string",
        "skw": false,
        "properties": {}
      },
      "deliverProjectId": {
        "typeKey": "integer",
        "is_relation": "N",
        "name": "交付项目ID",
        "typeName": "整型",
        "range": "",
        "default_value": "",
        "id": "deliverProjectId",
        "type": "integer",
        "skw": false,
        "properties": {}
      },
      "description": {
        "typeKey": "string",
        "is_relation": "N",
        "rangeLength": "",
        "name": "备注",
        "pattern": "",
        "typeName": "字符型",
        "default_value": "",
        "id": "description",
        "type": "string",
        "skw": false,
        "properties": {}
      },
      "envId": {
        "typeKey": "integer",
        "is_relation": "N",
        "name": "主环境ID",
        "typeName": "整型",
        "range": "",
        "default_value": "",
        "id": "envId",
        "type": "integer",
        "skw": false,
        "properties": {}
      },
      "env": {
        "typeKey": "string",
        "is_relation": "N",
        "rangeLength": "",
        "name": "环境名",
        "pattern": "",
        "typeName": "字符型",
        "default_value": "",
        "id": "env",
        "type": "string",
        "skw": false,
        "properties": {}
      },
      "platform": {
        "typeKey": "string",
        "is_relation": "N",
        "rangeLength": "",
        "name": "平台",
        "pattern": "",
        "typeName": "字符型",
        "default_value": "",
        "id": "platform",
        "type": "string",
        "skw": false,
        "properties": {}
      },
      "expiredTime": {
        "typeKey": "string",
        "is_relation": "N",
        "name": "过期时间",
        "format": "date-time",
        "typeName": "时间",
        "default_value": "",
        "id": "expiredTime",
        "type": "string",
        "skw": false,
        "properties": {}
      },
      "companyId": {
        "typeKey": "integer",
        "is_relation": "N",
        "name": "公司ID",
        "typeName": "整型",
        "range": "",
        "default_value": "",
        "id": "companyId",
        "type": "integer",
        "skw": false,
        "properties": {}
      },
      "zone": {
        "typeKey": "string",
        "is_relation": "N",
        "rangeLength": "",
        "name": "可用区",
        "pattern": "",
        "typeName": "字符型",
        "default_value": "",
        "id": "zone",
        "type": "string",
        "skw": false,
        "properties": {}
      },
      "regionId": {
        "typeKey": "string",
        "is_relation": "N",
        "rangeLength": "",
        "name": "区域",
        "pattern": "",
        "typeName": "字符型",
        "default_value": "",
        "id": "regionId",
        "type": "string",
        "skw": false,
        "properties": {}
      },
      "name": {
        "typeKey": "string",
        "is_relation": "N",
        "rangeLength": "",
        "name": "名称",
        "pattern": "",
        "typeName": "字符型",
        "default_value": "",
        "id": "name",
        "type": "string",
        "skw": false,
        "properties": {}
      },
      "isCustom": {
        "typeKey": "boolean",
        "is_relation": "N",
        "name": "是否自定义",
        "typeName": "布尔值",
        "id": "isCustom",
        "type": "boolean",
        "skw": false,
        "properties": {}
      },
      "createdTime": {
        "typeKey": "string",
        "is_relation": "N",
        "name": "创建时间",
        "format": "date-time",
        "typeName": "时间",
        "default_value": "",
        "id": "createdTime",
        "type": "string",
        "skw": false,
        "properties": {}
      },
      "id": {
        "typeKey": "integer",
        "is_relation": "N",
        "name": "id",
        "typeName": "整型",
        "range": "",
        "default_value": "",
        "id": "id",
        "type": "integer",
        "skw": false,
        "properties": {}
      },
      "projectName": {
        "typeKey": "string",
        "is_relation": "N",
        "rangeLength": "",
        "name": "项目名",
        "pattern": "",
        "typeName": "字符型",
        "default_value": "",
        "id": "projectName",
        "type": "string",
        "skw": false,
        "properties": {}
      },
      "projectId": {
        "typeKey": "integer",
        "is_relation": "N",
        "name": "主项目ID",
        "typeName": "整型",
        "range": "",
        "default_value": "",
        "id": "projectId",
        "type": "integer",
        "skw": false,
        "properties": {}
      },
      "account": {
        "typeKey": "string",
        "is_relation": "N",
        "rangeLength": "",
        "name": "账号",
        "pattern": "",
        "typeName": "字符型",
        "default_value": "",
        "id": "account",
        "type": "string",
        "skw": false,
        "properties": {}
      },
      "status": {
        "typeKey": "integer",
        "is_relation": "N",
        "name": "状态,1为正常",
        "typeName": "整型",
        "range": "",
        "default_value": "",
        "id": "status",
        "type": "integer",
        "skw": false,
        "properties": {}
      }
    },
    "searchConfig": {
      "searchTags": false,
      "ruleConfig": {},
      "searchSupport": false
    }
  },
  "tagSchema": {},
  "created": "2024-04-08 17:42:21",
  "updated": "2024-04-23 14:45:36",
  "changedBy": "yonghua.liu",
  "modelKey": "RTM",
  "modelName": "RTM",
  "modelId": 134,
  "canTransferEvent": null

""",
            },
            {"role": "cmdb", "content": "How many property in RTM_PRODUCT config?"},
            {"role": "cmdb", "content": "RTM_PRODUCT searchConfig?"},
            {"role": "cmdb", "content": "RTM_PRODUCT unique_key_rule?"},
        ]
    ]
dialog_chat(dialogs_temp)

cmdb: RTM_PRODUCT config

Assistant: RTM_PRODUCT config:
                
  "key": "RTM_PRODUCT",
  "name": "RTM产品线",
  "description": "RTM_PRODUCT",
  "attrValid": null,
  "filter": {},
  "metaSchema": {
    "unique_key_rule": [
      "platform",
      "account",
      "resourceInstanceId"
    ],
    "icon": "type1",
    "autoKey": "id",
    "subscribeConfig": {
      "subscribeItems": [],
      "isPublish": false
    },
    "orgAuthorization": {
      "enable": false
    },
    "required": [
      "account",
      "companyId",
      "platform",
      "regionId",
      "resourceInstanceId",
      "name"
    ],
    "properties": {
      "resourceInstanceId": {
        "typeKey": "string",
        "is_relation": "N",
        "rangeLength": "",
        "name": "唯一实例ID",
        "pattern": "",
        "typeName": "字符型",
        "default_value": "",
        "id": "resourceInstanceId",
        "type": "string",
        "skw": false,
        "properties": {}
      },
      "instanceName": {


In [27]:
def dialog_chat_role_cmdb(content: str):
    dialogs_temp:[Dialog] = [ [{"role": "cmdb", "content": content}]]
    dialog_chat(dialogs_temp)

In [31]:
print(datetime.datetime.now())
dialog_chat_role_cmdb("RTM_PRODUCT config modelKey is?")
print(datetime.datetime.now())

2024-05-12 12:17:58.896544
cmdb: RTM_PRODUCT config modelKey is?

> Assistant: A technical question!

In the context of RTM (Release to Manufacturing) products, the `config` modelKey is typically used to identify a specific product configuration.

In the RTM product configuration model, `modelKey` is a unique identifier that represents a specific product configuration. It's used to differentiate between different product configurations, such as different variants, options, or bundles.

The `modelKey` is usually a unique string that is assigned to each product configuration, and it's used to identify the configuration in various systems, such as product information management (PIM) systems, enterprise resource planning (ERP) systems, or customer relationship management (CRM) systems.

For example, a product configuration might have a `modelKey` of "ABC123", which identifies a specific product with specific features, options, and attributes. This `modelKey` can then be used to retrieve i

In [33]:
print(datetime.datetime.now())
dialog_chat_role_cmdb("RTM_PRODUCT name is?")
print(datetime.datetime.now())

2024-05-12 12:29:12.505233
cmdb: RTM_PRODUCT name is?

> Assistant: RTM stands for Release to Manufacturing, and it's a critical milestone in the software development process.

In RTM, the final product is prepared for mass production and distribution to customers. This typically involves a series of checks and tests to ensure the product meets the required quality and functionality standards.

So, the RTM PRODUCT name is... (drumroll please)... the same as the original product name!


2024-05-12 12:29:53.025960
